**Общая информация**

**Срок сдачи:** 21 октября 2019, 08:30 

**Штраф за опоздание:** по 0.5 балла за 24 часа задержки. Через 10 дней домашнее задание сгорает.

При отправлении ДЗ указывайте фамилию в названии файла
Присылать ДЗ необходимо в виде ссылки на свой github репозиторий на почту ml1.sphere@mail.ru с указанием темы в следующем формате:

[ML0919, Задание 1] Фамилия Имя.

Используйте данный Ipython Notebook при оформлении домашнего задания.


**Штрафные баллы:**

1. Невыполнение PEP8 -0.5 баллов
2. Отсутствие фамилии в имени скрипта (скрипт должен называться по аналогии со stroykova_hw1.ipynb) -0.5 баллов
3. Все строчки должны быть выполнены. Нужно, чтобы output команды можно было увидеть уже в git'е. В противном случае -0.5 баллов


In [1]:
import copy
import itertools
import re
import numpy as np
from scipy.sparse import csr_matrix
from scipy.stats import  mode
import matplotlib.pyplot as plt

import seaborn as sns
from sklearn import datasets
from sklearn.base import ClassifierMixin
from sklearn.datasets import fetch_mldata, fetch_20newsgroups

from sklearn.neighbors.base import NeighborsBase, KNeighborsMixin, SupervisedIntegerMixin 
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier, KDTree

from sklearn.model_selection import cross_validate, KFold

from sklearn.metrics.pairwise import euclidean_distances

%load_ext pycodestyle_magic

##### Задание 1 (1 балл)
Реализовать KNN в классе MyKNeighborsClassifier (обязательное условие: точность не ниже sklearn реализации)
Разберитесь самостоятельно, какая мера расстояния используется в KNeighborsClassifier дефолтно и реализуйте свой алгоритм именно с этой мерой. Самостоятельно разберитесь, как считается score из KNeighborsClassifier и реализуйте аналог в своём классе. Score не должен уступать значению KNN из sklearn

In [2]:
# %%pycodestyle


class MyKNeighborsClassifier(NeighborsBase, KNeighborsMixin,
                             SupervisedIntegerMixin, ClassifierMixin):

    def __init__(self, n_neighbors, algorithm='brute'):

        self.n_neighbors = n_neighbors
        self.algorithm = algorithm

    def fit(self, X, y):

        if self.n_neighbors < X.shape[0]:
            self.X_train = X
            self.y_train = np.array(y)
            self.classes = np.unique(y)

        else:
            print("Parameter n_neingbors is higher than the \
                    number of observations")
            return

    def predict(self, X):

        n_neighbors = self.n_neighbors
        algorithm = self.algorithm

        if n_neighbors < X.shape[0]:
            if algorithm == 'brute':
                neigh_ind, neigh_dist = self.kneighbors_brute(X)
            elif algorithm == 'kd_tree':
                tree = KDTree(self.X_train, leaf_size=2)
                neigh_dist, neigh_ind = tree.query(X, k=n_neighbors)
            else:
                print("The name of algorithm is incorrect. \
                Please, use 'brute' or 'kd_tree' ")
            y_pred = np.transpose(mode(self.y_train[neigh_ind], axis=1)[0])[0]
            return y_pred
        else:
            print("The number of observations in \
                  test set is higher than n_neingbors")

    def kneighbors_brute(self, X):

        n_neighbors = self.n_neighbors

        distances = euclidean_distances(X, self.X_train)
        # indices of closes n_neighbors elements
        sort_indices = distances.argsort(axis=1)[:, :n_neighbors]
        # sorted distances
        distances.sort()
        distances[:, :n_neighbors]

        return sort_indices, distances[:, :n_neighbors]

    def predict_proba(self, X):

        n_neighbors = self.n_neighbors
        classes = self.classes
        algorithm = self.algorithm

        if n_neighbors < X.shape[0]:

            if algorithm == 'brute':
                neigh_ind, neigh_dist = self.kneighbors_brute(X)
            elif algorithm == 'kd_tree':
                tree = KDTree(self.X_train, leaf_size=2)
                neigh_dist, neigh_ind = tree.query(X, k=n_neighbors)
            else:
                print("The name of algorithm is incorrect. \
                Please, use 'brute' or 'kd_tree' ")

            y_pred = self.y_train[neigh_ind]

            probabilities = np.count_nonzero(y_pred == classes[0], axis=1)

            for i in range(1, len(classes)):
                probabilities = np.vstack(
                    (probabilities, np.count_nonzero(
                        y_pred == classes[i], axis=1)))

            probabilities = probabilities/probabilities.sum(axis=0)

            return np.transpose(probabilities)

        else:
            print("The number of observations in \
                  test set is higher than n_neingbors")

    def score(self, X, y):

        y_pred = self.predict(X)

        differing_labels = np.count_nonzero(y_pred - y)

        return 1 - differing_labels / y_pred.shape[0]

**IRIS**

В библиотеке scikit-learn есть несколько датасетов из коробки. Один из них [Ирисы Фишера](https://ru.wikipedia.org/wiki/%D0%98%D1%80%D0%B8%D1%81%D1%8B_%D0%A4%D0%B8%D1%88%D0%B5%D1%80%D0%B0)

In [3]:
iris = datasets.load_iris()

In [4]:
X_train, X_test, y_train, y_test = train_test_split(
    iris.data, iris.target, test_size=0.1, stratify=iris.target)

In [5]:
clf = KNeighborsClassifier(n_neighbors=2, algorithm='brute')
my_clf = MyKNeighborsClassifier(n_neighbors=2, algorithm='brute')

In [6]:
clf.fit(X_train, y_train)
my_clf.fit(X_train, y_train)

In [7]:
assert abs(my_clf.score(X_test, y_test) - clf.score(X_test, y_test))<0.005, "Score must be simillar"

**Задание 2 (0.5 балла)**

Давайте попробуем добиться скорости работы на fit, predict и predict_proba сравнимой со sklearn для iris.
Для этого используем numpy

In [8]:
%time clf.fit(X_train, y_train)

CPU times: user 1.62 ms, sys: 0 ns, total: 1.62 ms
Wall time: 1.17 ms


KNeighborsClassifier(algorithm='brute', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=2, p=2,
                     weights='uniform')

In [9]:
%time my_clf.fit(X_train, y_train)

CPU times: user 239 µs, sys: 39 µs, total: 278 µs
Wall time: 225 µs


In [10]:
%time clf.predict(X_test)

CPU times: user 5.07 ms, sys: 177 µs, total: 5.24 ms
Wall time: 4.76 ms


array([2, 0, 1, 2, 0, 2, 2, 1, 2, 2, 1, 0, 0, 0, 1])

In [11]:
%time my_clf.predict(X_test)

CPU times: user 8.05 ms, sys: 7 µs, total: 8.06 ms
Wall time: 6.17 ms


array([2, 0, 1, 2, 0, 2, 2, 1, 2, 2, 1, 0, 0, 0, 1])

In [12]:
%time clf.predict_proba(X_test)

CPU times: user 2.46 ms, sys: 0 ns, total: 2.46 ms
Wall time: 1.8 ms


array([[0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.]])

In [13]:
%time my_clf.predict_proba(X_test)

CPU times: user 2.91 ms, sys: 0 ns, total: 2.91 ms
Wall time: 4.66 ms


array([[0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.]])

###### Задание 3 (1 балл)
Добавьте algorithm='kd_tree' в реализацию KNN (использовать KDTree из sklearn.neighbors). Необходимо добиться скорости работы на fit,  predict и predict_proba сравнимой со sklearn для iris.
Для этого используем numpy. Score не должен уступать значению KNN из sklearn

In [14]:
clf = KNeighborsClassifier(n_neighbors=2, algorithm='kd_tree')
my_clf = MyKNeighborsClassifier(n_neighbors=2, algorithm='kd_tree')

In [15]:
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.1, stratify=iris.target)

In [16]:
%time clf.fit(X_train, y_train)

CPU times: user 1.5 ms, sys: 0 ns, total: 1.5 ms
Wall time: 1.35 ms


KNeighborsClassifier(algorithm='kd_tree', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=2, p=2,
                     weights='uniform')

In [17]:
%time my_clf.fit(X_train, y_train)

CPU times: user 135 µs, sys: 22 µs, total: 157 µs
Wall time: 167 µs


In [18]:
%time clf.predict(X_test)

CPU times: user 7.73 ms, sys: 0 ns, total: 7.73 ms
Wall time: 5.81 ms


array([1, 0, 0, 1, 2, 2, 2, 0, 2, 2, 2, 1, 0, 0, 1])

In [19]:
%time my_clf.predict(X_test)

CPU times: user 4.49 ms, sys: 83 µs, total: 4.57 ms
Wall time: 3.42 ms


array([1, 0, 0, 1, 2, 2, 2, 0, 2, 2, 2, 1, 0, 0, 1])

In [20]:
%time clf.predict_proba(X_test)

CPU times: user 1.74 ms, sys: 180 µs, total: 1.92 ms
Wall time: 1.68 ms


array([[0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.]])

In [21]:
%time my_clf.predict_proba(X_test)

CPU times: user 1.37 ms, sys: 0 ns, total: 1.37 ms
Wall time: 1.07 ms


array([[0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.]])

In [22]:
assert abs(my_clf.score(X_test, y_test) - clf.score(X_test,y_test))<0.005, "Score must be simillar"

**Задание 4 (2.5 балла)**

Рассмотрим новый датасет 20 newsgroups

In [23]:
newsgroups = fetch_20newsgroups(subset='train',remove=['headers','footers', 'quotes'])

In [24]:
data = newsgroups['data']
target = newsgroups['target']

Переведите во всех документах все буквы в нижний регистр и замените во всех документах символы, не
являющиеся буквами и цифрами, на пробелы. Далее разбейте текста по пробельным символам на токены(термы/слова). Удалите текста, содержащие только пробелы.

In [25]:
# %%pycodestyle
def tokenize(file_str):
    '''
    file_str - str
    '''
    file_str = file_str.lower()
    file_str = re.sub(r"[^a-zA-Z0-9]", " ", file_str)

    file_l = file_str.split()

    return file_l

In [26]:
# %%pycodestyle
data_tok1 = [tokenize(l) for l in copy.deepcopy(data)]

data_tok = []
target_new = []

for line, y in zip(data_tok1, target):
    if line != []:
        data_tok.append(line)
        target_new.append(y)

In [27]:
len(data_tok)

11004

In [28]:
assert all(isinstance(row, (list, tuple)) for row in data_tok), "please convert each line into a list of tokens (strings)"
assert all(all(isinstance(tok, str) for tok in row) for row in data_tok), "please convert each line into a list of tokens (strings)"
is_latin = lambda tok: all('a' <= x.lower() <= 'z' for x in tok)
assert all(map(lambda l: not is_latin(l) or l.islower() , map(' '.join, data_tok))), "please make sure that you lowercase the data and drop spaced texts"

Преобразуйте датасет в разреженную матрицу scipy.sparse.csr_matrix, где значение x в позиции (i, j)
означает, что в документе i слово j встретилось x раз

In [29]:
indptr = [0]
indices = []
data = []
vocabulary = {}

for d in data_tok:
    for term in d:
        if (len(term) < 3) or (len(term) > 20) or not term.isdigit():
            continue
        else:
            index = vocabulary.setdefault(term, len(vocabulary))
            indices.append(index)
            data.append(1)
    indptr.append(len(indices))

word_count_m = csr_matrix((data, indices, indptr), dtype=int).toarray()

In [30]:
word_count_m.shape

(11004, 4393)

In [31]:
condition1 = word_count_m.sum(axis=0) < 100
# condition1 = condition1[np.newaxis, :]

condition2 = word_count_m.sum(axis=0) > 2
# condition2 = condition2[np.newaxis, :]
np.count_nonzero(condition1), np.count_nonzero(condition2)

(4369, 1463)

In [32]:
word_count_m = word_count_m[:,np.logical_or(condition1, condition2)]

In [33]:
word_count_m.shape

(11004, 4393)

*Так мы получили векторное представление наших текстов. Значит можно приступать к задаче обучения модели*

Реализуйте разбиение выборки для кросс-валидации на 3 фолдах. Разрешено использовать sklearn.cross_validation

In [34]:
my_clf = MyKNeighborsClassifier(n_neighbors=2, algorithm='brute')

In [35]:
my_clf.fit(word_count_m, target_new)

In [36]:
my_clf.X_train.shape, my_clf.y_train.shape

((11004, 4393), (11004,))

In [37]:
scores = cross_validate(my_clf, word_count_m, target_new, cv=3)

In [38]:
np.mean(scores['test_score'])

0.09223998514602942

Напишите метод, позволяющий найти оптимальное количество ближайших соседей(дающее максимальный score в среднем на валидации на 3 фолдах).
Постройте график зависимости среднего score от количества соседей. Можно рассмотреть число соседей от 1 до 10.

Как изменится качество на валидации, если:

1. Используется косинусная метрика вместо евклидовой.
2. К текстам применяется TfIdf преобразование( sklearn.feature_extraction.text.TfidfTransformer)

Сравните модели, выберите лучшую.

Загрузим  теперь test  часть нашей выборки и преобразуем её аналогично с train частью. Не забудьте, что наборы слов в train и test части могут отличаться.

In [39]:
newsgroups = fetch_20newsgroups(subset='test',remove=['headers','footers', 'quotes'])

Оценим качество(score) вашей лучшей модели на test части датасета. Отличается ли оно от кросс-валидации? Попробуйте сделать выводы, почему отличается качество.